# CW2: Hospital Readmission Prediction for Diabetic Patients


### Imports


In [1]:
# Basics
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


## Task 1: Dataset description


##  Task 2: Data assembling and initial pre-processing (15%)
